<a href="https://colab.research.google.com/github/KAnejjar/python_projects/blob/main/Human%20Image%20Generation%20with%20Animal%20Feature%20/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/justinpinkney/stable-diffusion.git
%cd stable-diffusion
!pip install --upgrade pip
!pip install -r requirements.txt

!pip install --upgrade keras # on lambda stack we need to upgrade keras
!pip uninstall -y torchtext # on colab we need to remove torchtext

Cloning into 'stable-diffusion'...
remote: Enumerating objects: 1755, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 1755 (delta 3), reused 5 (delta 3), pack-reused 1747 (from 1)
Receiving objects: 100% (1755/1755), 73.93 MiB | 18.33 MiB/s, done.
Resolving deltas: 100% (1082/1082), done.
/content/stable-diffusion
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
Obtaining taming-transformers from git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers (from -r requirements.txt (line 23))
  Cloning https://github.com/CompVis/taming-transformers.git (to revision master) to ./src/taming-transformers
  Running command git clone --filter=blob:none -

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0


In [ ]:
!nvidia-smi

Thu Dec 12 10:26:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import os
from PIL import Image



def preprocess_images(input_dir, output_dir, size=(256, 256)):
    os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist
    for file_name in os.listdir(input_dir):  # Iterate over files in the input directory
        img_path = os.path.join(input_dir, file_name)
        img = Image.open(img_path).convert("RGB")  # Open the image and convert to RGB
        img = img.resize(size)  # Resize the image
        img.save(os.path.join(output_dir, file_name))  # Save the preprocessed image to output_dir

# Use the correct dataset path
!ls /content/my_dataset/images/

input_path = "/content/my_dataset/images/"  # Update with your dataset path
output_path = "/content/my_dataset/preprocessed_images"  # Output directory
preprocess_images(input_path, output_path)  # Run the preprocessing function

image10.jpg  image14.jpg  image18.jpg  image21.jpg  image25.jpg  image3.jpg  image7.jpg
image11.jpg  image15.jpg  image19.jpg  image22.jpg  image26.jpg  image4.jpg  image8.jpg
image12.jpg  image16.jpg  image1.jpg   image23.jpg  image27.jpg  image5.jpg  image9.jpg
image13.jpg  image17.jpg  image20.jpg  image24.jpg  image2.jpg	 image6.jpg


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [9]:
import os
from PIL import Image
from datasets import Dataset
import numpy as np
from torch.utils.data import DataLoader

# Preprocess images: resize and save to output directory
def preprocess_images(input_dir, output_dir, size=(256, 256)):
    os.makedirs(output_dir, exist_ok=True)  # Create output directory if it doesn't exist
    for file_name in os.listdir(input_dir):  # Iterate over files in input directory
        img_path = os.path.join(input_dir, file_name)
        img = Image.open(img_path).convert("RGB")  # Open image and convert to RGB
        img = img.resize(size)  # Resize the image
        img.save(os.path.join(output_dir, file_name))  # Save preprocessed image to output_dir

# Paths for input, output, and captions
input_path = "/content/my_dataset/images/"  # Directory containing raw images
output_path = "/content/my_dataset/preprocessed_images/"  # Directory for resized images
captions_path = "/content/my_dataset/captions.txt"  # Path to captions file

# Preprocess images
preprocess_images(input_path, output_path)

# Ensure captions.txt exists
assert os.path.exists(captions_path), "The captions.txt file is missing!"

# Load captions and prepare dataset
data = []
with open(captions_path, "r") as f:
    for line in f:
        image_name, caption = line.strip().split("   ")  # Assuming '   ' separates filename and caption
        image_path = os.path.join(output_path, image_name)
        if os.path.exists(image_path):  # Ensure image exists
            data.append({"image": image_path, "text": caption})

# Convert list of dictionaries to a dictionary of lists for Hugging Face Dataset
data_dict = {
    "image": [item["image"] for item in data],
    "text": [item["text"] for item in data],
}

# Create Hugging Face Dataset
ds = Dataset.from_dict(data_dict)

# Preprocess dataset to load images as NumPy arrays
def preprocess_data(sample):
    if isinstance(sample["image"], str):  # Ensure image path is valid
        image = Image.open(sample["image"]).convert("RGB")
        image = image.resize((256, 256))  # Resize image
        sample["image"] = np.array(image)  # Convert to NumPy array
    else:
        raise ValueError(f"Invalid image format: {type(sample['image'])}")
    return sample

# Map the preprocessing function to the dataset
ds = ds.map(preprocess_data, batched=False)

# Debug: Check if all images are NumPy arrays
for i in range(3):  # Check first 3 items
    print(f"Sample {i} image type: {type(ds[i]['image'])}")  # Should print: <class 'numpy.ndarray'>

# Define a simple collate function for DataLoader
def collate_fn(batch):
    # Stack image arrays and collect captions
    images = np.stack([item["image"] for item in batch])  # Stack NumPy arrays
    captions = [item["text"] for item in batch]  # Collect captions
    return {"images": images, "captions": captions}

# Create DataLoader
dataloader = DataLoader(ds, batch_size=8, shuffle=True, collate_fn=collate_fn)

# Test DataLoader
for batch in dataloader:
    print("Images shape:", batch["images"].shape)  # Should be [batch_size, 256, 256, 3]
    print("Captions:", batch["captions"])
    break


Map:   0%|          | 0/27 [00:00<?, ? examples/s]

Sample 0 image type: <class 'list'>
Sample 1 image type: <class 'list'>
Sample 2 image type: <class 'list'>
Images shape: (8, 256, 256, 3)
Captions: ['An Asian girl with long brown hair and brown eyes, wearing a hoodie. She has big fluffy blond cat ears with soft white inner fur.', 'A brunette female with subtle beige-furred cat ears and warm brown eyes, wearing a casual sweater. The background is a simple gradient that highlights her features.', 'A blonde female with light-brown cat ears and piercing blue eyes, styled with a fitted beige cardigan. The scene is softly lit in an urban outdoor setting.', 'A young woman with short dark brown hair and bright blue eyes, wearing a blue jacket over a white shirt. She has prominent feline ears covered in brown fur with lighter beige inner fur and black tips, adding a striking animalistic feature to her appearance.', 'A blonde girl with long golden-blond hair, blue eyes, and a bright expression, wearing a beige cardigan over a white shirt. She 

In [11]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [12]:
!pip install diffusers transformers accelerate

In [14]:
# Install required dependencies
!pip install --upgrade diffusers transformers accelerate datasets huggingface_hub xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 68.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.1.1
    Uninstalling accelerate-1.1.1:
      Successfully uninstalled accelerate-1.1.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


In [18]:
# Install required dependencies
!pip install --upgrade diffusers transformers accelerate datasets huggingface_hub xformers

import os
from PIL import Image
from datasets import Dataset
import numpy as np
from torch.utils.data import DataLoader
from diffusers import UNet2DConditionModel, AutoencoderKL, DDPMScheduler
from transformers import CLIPTextModel, CLIPTokenizer
from torch.optim import AdamW
from accelerate import Accelerator
import torch

# Step 1: Prepare Dataset
# Define paths
image_dir = "/content/my_dataset/preprocessed_images"
caption_file = "/content/my_dataset/captions.txt"

# Load captions
data = []
with open(caption_file, "r") as f:
    for line in f:
        image_name, caption = line.strip().split("   ")  # Assuming '   ' separates filename and caption
        image_path = os.path.join(image_dir, image_name)
        if os.path.exists(image_path):
            data.append({"image": image_path, "text": caption})

# Create Hugging Face Dataset
ds = Dataset.from_dict({
    "image": [item["image"] for item in data],
    "text": [item["text"] for item in data],
})

# Preprocess images
def preprocess_images(sample):
    image = Image.open(sample["image"]).convert("RGB")
    image = image.resize((512, 512))  # Resize to 512x512 for Stable Diffusion XL
    sample["image"] = np.array(image)
    return sample

ds = ds.map(preprocess_images, batched=False)

# Step 2: Define Training Parameters
model_id = "stabilityai/stable-diffusion-xl-base-1.0"
num_epochs = 3
train_batch_size = 4
learning_rate = 5e-6
gradient_accumulation_steps = 2
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load Stable Diffusion XL Components
unet = UNet2DConditionModel.from_pretrained(model_id, subfolder="unet").to(device)
vae = AutoencoderKL.from_pretrained(model_id, subfolder="vae").to(device)
text_encoder = CLIPTextModel.from_pretrained(model_id, subfolder="text_encoder").to(device)
tokenizer = CLIPTokenizer.from_pretrained(model_id, subfolder="tokenizer")
noise_scheduler = DDPMScheduler.from_pretrained(model_id, subfolder="scheduler")

# Step 3: Prepare DataLoader
def collate_fn(batch):
    images = torch.tensor([item["image"] for item in batch]).permute(0, 3, 1, 2) / 255.0  # Normalize
    captions = [item["text"] for item in batch]
    return {"images": images.to(device), "captions": captions}

dataloader = DataLoader(ds, batch_size=train_batch_size, shuffle=True, collate_fn=collate_fn)

# Step 4: Set Up Optimizer and Accelerator
optimizer = AdamW(unet.parameters(), lr=learning_rate)
accelerator = Accelerator(mixed_precision="fp16")
unet, optimizer, dataloader = accelerator.prepare(unet, optimizer, dataloader)

# Step 5: Training Loop
for epoch in range(num_epochs):
    unet.train()
    for step, batch in enumerate(dataloader):
        images = batch["images"]
        captions = batch["captions"]

        # Tokenize captions
        inputs = tokenizer(captions, padding="max_length", truncation=True, max_length=77, return_tensors="pt").to(device)
        encoder_hidden_states = text_encoder(inputs.input_ids)[0]

        # Add noise to images
        noise = torch.randn_like(images).to(device)
        timesteps = torch.randint(0, noise_scheduler.num_train_timesteps, (images.shape[0],), device=device).long()
        noisy_images = noise_scheduler.add_noise(images, noise, timesteps)

        # Predict noise
        noise_pred = unet(noisy_images, timesteps, encoder_hidden_states).sample

        # Compute loss
        loss = torch.nn.functional.mse_loss(noise_pred, noise)
        accelerator.backward(loss)

        # Optimizer step
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        # Log progress
        if step % 10 == 0:
            print(f"Epoch {epoch}, Step {step}, Loss: {loss.item()}")

# Step 6: Save Fine-Tuned Model
unet.save_pretrained("/content/stable-diffusion-xl-finetuned/unet")
text_encoder.save_pretrained("/content/stable-diffusion-xl-finetuned/text_encoder")



Map:   0%|          | 0/27 [00:00<?, ? examples/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 3.06 MiB is free. Process 9956 has 14.74 GiB memory in use. Of the allocated memory 14.30 GiB is allocated by PyTorch, and 321.50 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Step 7: Test Fine-Tuned Model
from diffusers import StableDiffusionXLImg2ImgPipeline

# Load Fine-Tuned Model
pipeline = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    unet="/content/stable-diffusion-xl-finetuned/unet",
    text_encoder="/content/stable-diffusion-xl-finetuned/text_encoder"
).to("cuda")

# Generate Images
prompt = "A futuristic city with flying cars"
generated_image = pipeline(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

# Display Image
generated_image.show()